<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Michelle Fong

Date: 16/3/2024


Environment: Python3, Google Colab

Libraries used:
* re (for regular expression, installed and imported)
* pandas (for data manipulation)
* json (for exporting file)
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Raw Data](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Transforming Data](#latin) <br>
[5. Dataframe Merging and Transforming](#write) <br>
$\;\;\;\;$[5.1. Merging Dataframe](#merge) <br>
$\;\;\;\;$[5.2. Data Cleaning](#clean) <br>
[6. Export to JSON](#export) <br>
[7. Summary](#summary) <br>
[8. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

The goal of this task is to read in a `TXT` file contains data in `XML` format and export into `JSON` file.   
Our approach to this task is to first split the data by `<assignment-entry>`, then extract the infomation as DataFrame for re-formatting or merging. The DataFrame would then be transferred into JSON file. The data file contains 6338 records of property assignment including assignor and assignees.


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** for data manipulation after reading the data as DataFrame
* **json:** for exporting dict into json file

In [ ]:
import re
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, our group is using `Group015.txt` under `Students data/Task1` as the input datafile. It contains informaiton about property assignment.


Having examined the file content, the following observations were made:
- Irrelevant data exists at the begining and at then end, they need to be removed. Only those with `<assignment-entry>` tag are relevant data.
- Some assignments lack values for certain columns, eg date-acknowledged, execution-date under assignor-info
- In assignor-info session, the country name is not stanardardized: some use United States while some use the states of the US

<div class="alert alert-block alert-warning">

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the goal is to define a regular expression pattern to parse and extract relevant information from text files. Initially, we observed irrelevant data at the beginning and the end of the text files. The relevant information starts with the tag <assignment-entry> and therefore it is used to indicate the start and the end of the relevant data and all data in between is extracted for further processing.

Once the relevant data is extracted, we replace XML special characters with readable characters.

The overall process involves:

1. Defining a regular expression pattern to identify and capture the relevant data starting from the <assignment-entry> tag.
2. Reading in the text files and applying the regular expression pattern to replace XML entities with readable characters
3. Extracting the corresponding information from the cleaned text and storing it in DataFrame for further analysis and processing.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Patterns for regular expressions are defined in the "Transforming Data" part.

We generally use `<tag>` and `</tag>` with `(.*?)` to get all the values between the tag names.

For tags which are under "assignors" and "assignees" tags:

*   Use `\s` to escape any spaces between  `<assignors>` \ `<assignees>`, and find the `<person-or-organization-name>` to grouping and get all values between `<person-or-organization-name> `and` <\person-or-organization-name>`.
*   Use `.*?` to escape all the characters between `<assignors>` \ `<assignees>` and `<target_tags>` to find the target tag, and get everything in group `(.*?)` between `<target_tags>` and `</target_tags>`



Some examples:


*   reel-no: `r'<reel-no>(.*?)</reel-no>'`
*   person-or-organization-name under **assignor**: `r'<assignor>\s*<person-or-organization-name>(.*?)</person-or-organization-name>'`
*   nationality under **assignee**: `r'<assignee>.*?<nationality>(.*?)</nationality>'`




-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

The txt file is read in this step with separation by `<assignment-entry>` and replaces XML characters using dictionary. 4 XML characters were joined as a pattern and being replaced using regex.  The data is ready for regular expression as the next step.

In [ ]:
path = '/content/drive/Shareddrives/FIT5196_S1_2024/A1/Students data/Task 1/Group015.txt'

with open(path, 'r') as f:
    relevant_data_started = False
    data = ""
    for line in f:
        if "<assignment-entry>" in line:
            relevant_data_started = True
        if relevant_data_started:
            data += line

# replace xml character
# dict mapping XML entities to their characters
xml_entities = {
    '&amp;': '&',
    '&quot;': '"',
    '&apos;': "'"
}

# func replacing XML entities using the dictionary
def replace_entities(match):
    return xml_entities[match.group(0)]
    # take the match as input
    # retrievem the whole substring
    # return corr chr

# construct pattern
pattern = '|'.join(re.escape(entity) for entity in xml_entities.keys())
    # escape any special chr in xml entities
    # join by |: \&amp;|\&quot;|\&apos;

# replace matched pattern with the chr (defined in dict)
data = re.sub(pattern, replace_entities, data)


In [ ]:
#checking data in first 100 characters
data[:100]

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Transforming Data <a class="anchor" name="latin"></a>

After reading in the data, it is extracted into dictionary key-value pairs to be exported in DataFrame format. They are extracted by finding the corresponsing pattern defined in previous session using `re`.

Some information were nested within other tags, requiring a multi-layered search for extraction. For example, `correspondent_party` was nested under `person-or-organization-name`, which was under `correspondent` within `assignment`section. In this case, they were matched layer by layer. `re.DOTALL` is used to match patterns across multiple lines. Otherwise, only the first occurance would be matched [1].

Similar process was repeated in extracing assignors and assignees information, except there might be 0 or more than 1 assignors and assignees. Using `re.DOTALL` ensures all assignors were captured. In cases of no assignors or assignees were found, the value would be replaced by NA.

Following the extraction process, the  information was stored in a DataFrame. Some columns such as date are reformatted to meet the required datatype and format. To avoid conversion errors of datetime objects, `errors='coerce'` was added to coerce invalid parsing [2].

Then, `reelno` and `frameno` columns were concate as `rt-id`, which will be the reference index for merging three dataframes.

In [ ]:
# <assignment>
# split by assignemnt tag
entries = data.split('</assignment-entry>')[:-1]
entry_dicts = []

# extract info
for entry in entries:
    # dictionary to store assignment attributes
    entry_dict = {}

    # extract reel-no
    reelno_match = re.search(r'<reel-no>(.*?)</reel-no>', entry)
        # (.*?): match any chr 0/Many times as few time as possible (?)
    if reelno_match:
        entry_dict['reelno'] = reelno_match.group(1) # first occurance

    # extract frame-no
    frameno_match = re.search(r'<frame-no>(.*?)</frame-no>', entry)
    if frameno_match:
        entry_dict['frameno'] = frameno_match.group(1)

    # last-update-date
    last_update_date_match = re.search(r'<last-update-date>(.*?)</last-update-date>', entry)
    if last_update_date_match:
        entry_dict['last_update_date'] = last_update_date_match.group(1)

    # extract conveyance-text
    conveyance_text_match = re.search(r'<conveyance-text>(.*?)</conveyance-text>', entry)
    if conveyance_text_match:
        entry_dict['conveyance_text'] = conveyance_text_match.group(1)

    # extract correspondent_party
    correspondent_match = re.search(r'<correspondent>(.*?)</correspondent>', entry, re.DOTALL) # flag to match any pattern inc newline chr
    if correspondent_match:
        correspondent_text = correspondent_match.group(1)

        # match person_organization_name within correspondent
        person_organization_name_match = re.search(r'<person-or-organization-name>(.*?)</person-or-organization-name>', correspondent_text, re.DOTALL)
        if person_organization_name_match:
            entry_dict['correspondent_party'] = person_organization_name_match.group(1)

    # extract properties and count them
    properties_match = re.search(r'<properties>(.*?)</properties>', entry, re.DOTALL)
    if properties_match:
        properties_text = properties_match.group(1)
        property_count = len(re.findall(r'<property>', properties_text))

        entry_dict['property_count'] = property_count

    entry_dicts.append(entry_dict)


# Convert into pd DataFrame
entry_df = pd.DataFrame(entry_dicts)

# Combine reelno and frameno as rf-id:
entry_df['rf-id'] = entry_df.apply(lambda row: f"{row['reelno']}{row['frameno']}", axis=1)
# rearrange col
entry_df = entry_df[['rf-id', 'reelno', 'frameno', 'last_update_date', 'conveyance_text', 'correspondent_party', 'property_count']]

# change last_update_date format
    # convert to datetime format
entry_df['last_update_date'] = pd.to_datetime(entry_df['last_update_date'], errors='coerce')

    # format as YYYY-MM-DD
entry_df['last_update_date'] = entry_df['last_update_date'].dt.strftime('%Y-%m-%d')


entry_df

In [ ]:
#print(entry_df.shape)
#print(type(entry_df))

In [ ]:
# <assignors>
# Create dictionary for saving assignors' info
assignor_dicts = []

# Get assignor data in different entries
for assignor in entries:
    # Create a dictionary to store assignment attributes
    assignor_dict = {}

    # Extract data from tags
    reelno_match = re.search(r'<reel-no>(.*?)</reel-no>', assignor)
    frameno_match = re.search(r'<frame-no>(.*?)</frame-no>', assignor)
    assignor_name_match = re.search(r'<assignor>\s*<person-or-organization-name>(.*?)</person-or-organization-name>', assignor, re.DOTALL)
    assignor_ack_date_match = re.search(r'<assignor>.*?<date-acknowledged>(.*?)</date-acknowledged>', assignor, re.DOTALL)
    assignor_exe_date_match = re.search(r'<execution-date>(.*?)</execution-date>', assignor)
    assignor_nation_match = re.search(r'<assignor>.*?<nationality>(.*?)</nationality>', assignor, re.DOTALL)
    assignor_legal_match = re.search(r'<assignee>.*?<legal-entity-text>(.*?)</legal-entity-text>', assignor, re.DOTALL)

    # Get values for referred key, fill "NA" if no matches
    assignor_dict['reelno'] = reelno_match.group(1) if reelno_match else "NA"
    assignor_dict['frameno'] = frameno_match.group(1) if frameno_match else "NA"
    assignor_dict['party-name'] = assignor_name_match.group(1) if assignor_name_match else "NA"
    assignor_dict['date-acknowledge'] = assignor_ack_date_match.group(1) if assignor_ack_date_match else "NA"
    assignor_dict['execution-date'] = assignor_exe_date_match.group(1) if assignor_exe_date_match else "NA"
    assignor_dict['nationality'] = assignor_nation_match.group(1) if assignor_nation_match else "NA"
    assignor_dict['legal-entity-text'] = assignor_legal_match.group(1) if assignor_legal_match else "NA"

    # Append data into dictionary
    assignor_dicts.append(assignor_dict)

# Convert the list of dictionaries to a pandas DataFrame
assignor_df = pd.DataFrame(assignor_dicts)

# Combine reelno & frameno
assignor_df['rf-id'] = assignor_df['reelno'] + assignor_df['frameno']

# Remove reelno & frameno columns
assignor_df = assignor_df[['rf-id', 'party-name', 'date-acknowledge', 'execution-date', 'nationality','legal-entity-text']]

#convert to datetime
assignor_df['date-acknowledge'] = pd.to_datetime(assignor_df['date-acknowledge'], errors='coerce')
assignor_df['execution-date'] = pd.to_datetime(assignor_df['execution-date'], errors='coerce')

# format as YYYY-MM-DD
assignor_df['date-acknowledge'] = assignor_df['date-acknowledge'].dt.strftime('%Y-%m-%d')
assignor_df['execution-date'] = assignor_df['execution-date'].dt.strftime('%Y-%m-%d')

#fill NA
assignor_df.fillna('NA', inplace=True)

# Display the DataFrame
assignor_df


In [ ]:
# <assignees>
# Create dictionary for saving assignees info
assignee_dicts = []

# Get assignor data in different entries
for assignee in entries:
    # Create a dictionary to store assignment attributes
    assignee_dict = {}

    # Extract data from tags
    reelno_match = re.search(r'<reel-no>(.*?)</reel-no>', assignee)
    frameno_match = re.search(r'<frame-no>(.*?)</frame-no>', assignee)
    assignee_name_match = re.search(r'<assignee>\s*<person-or-organization-name>(.*?)</person-or-organization-name>', assignee, re.DOTALL)
    assignee_nation_match = re.search(r'<assignee>.*?<nationality>(.*?)</nationality>', assignee, re.DOTALL)
    assignee_legal_match = re.search(r'<assignee>.*?<legal-entity-text>(.*?)</legal-entity-text>', assignee, re.DOTALL)

    # Get values for referred key, fill "NA" if no matches
    assignee_dict['reelno'] = reelno_match.group(1) if reelno_match else "NA"
    assignee_dict['frameno'] = frameno_match.group(1) if frameno_match else "NA"
    assignee_dict['party-name'] = assignee_name_match.group(1) if assignee_name_match else "NA"
    assignee_dict['nationality'] = assignee_nation_match.group(1) if assignee_nation_match else "NA"
    assignee_dict['legal-entity-text'] = assignee_legal_match.group(1) if assignee_legal_match else "NA"

    # Append data into dictionary
    assignee_dicts.append(assignee_dict)

# Convert the list of dictionaries to a pandas DataFrame
assignee_df = pd.DataFrame(assignee_dicts)

# Combine reelno & frameno
assignee_df['rf-id'] = assignee_df['reelno'] + assignee_df['frameno']

# Remove reelno & frameno columns
assignee_df = assignee_df[['rf-id', 'party-name', 'nationality', 'legal-entity-text']]

# Display the DataFrame
assignee_df


In [ ]:
# checking shapes in different dataframe
print(f"entry_df shape:{entry_df.shape}")
print(f"assignor_df shape:{assignor_df.shape}")
print(f"assignee_df shape:{assignee_df.shape}")

In [ ]:
# cheching if all 3 are in dataframe format
print(type(entry_df))
print(type(assignor_df))
print(type(assignee_df))

-------------------------------------

<div class="alert alert-block alert-warning">

## 5.  Dataframe Merging and Transforming <a class="anchor" name="write"></a>

</div>

In this session, the objective was to merge three dataframes created in previous sessions using the reference column rf-id.

However, during the process, anomalies were found in the countries name. Specifically, state names were used instead of country names for three countries: the United States, the United Kingdom, and Canada. To solve this, we created 3 lists of states and replaced the state names with the corresponding country names.

It was noticed that title were included in the `person-or-organization-name` regardless of the legal entity role. We decided to remove titles for individuals while retaining titles for companies.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Merging dataframe <a class="anchor" name="merge"></a>

In [ ]:
# Merging df
assignment_df = pd.merge(entry_df, assignor_df, on='rf-id', how='left')
assignment_df = pd.merge(assignment_df, assignee_df, on='rf-id', how='left')
assignment_df.head(5)

<div class="alert alert-block alert-info">
    
### 5.2. Data Cleaning <a class="anchor" name="clean"></a>

In [ ]:
# check country unique value
# assignment_df['nationality_x'].unique()

# usa states list
usa_list = [ 'ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT',
            'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA',
             'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN',
             'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
             'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA',
             'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS',
             'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING', 'UNITED STATES']

# uk list
uk_list = ['ENGLAND', 'SCOTLAND', 'WALES', 'NORTHERN IRELAND', 'UNITED KINGDOM']

# canada list
ca_list = [ 'ALBERTA', 'BRITISH COLUMBIA', 'MANITOBA', 'NEW BRUNSWICK', 'NEWFOUNDLAND AND LABRADOR',
           'NOVA SCOTIA', 'ONTARIO', 'PRINCE EDWARD ISLAND', 'QUEBEC',
            'SASKATCHEWAN', 'NORTHWEST TERRITORIES', 'NUNAVUT', 'YUKON' ]

# revise nationality_x value into USA & UK
assignment_df['nationality_x'] = assignment_df['nationality_x'].replace(usa_list, 'USA')
assignment_df['nationality_x'] = assignment_df['nationality_x'].replace(uk_list, 'UK')
assignment_df['nationality_x'] = assignment_df['nationality_x'].replace(ca_list, 'CANADA')
assignment_df['nationality_x'] = assignment_df['nationality_x'].replace('STATELESS', 'NA') # replace 'STATELESS into NA'

# revise nationality_y value into USA & UK
assignment_df['nationality_y'] = assignment_df['nationality_y'].replace(usa_list, 'USA')
assignment_df['nationality_y'] = assignment_df['nationality_y'].replace(uk_list, 'UK')
assignment_df['nationality_y'] = assignment_df['nationality_y'].replace(ca_list, 'CANADA')
assignment_df['nationality_y'] = assignment_df['nationality_y'].replace('STATELESS', 'NA') # replace 'STATELESS into NA

# check df
print(f"assignment_df shape:{assignment_df.shape}")
assignment_df

In [ ]:
titles = ["Mr", "Mrs", "Miss", "Ms", "Mx", "Sir", "Dame", "Dr", "Cllr", "Lady", "Lord"]
assignment_df.loc[assignment_df['legal-entity-text_x'] == 'INDIV', 'party-name_y'] = assignment_df.loc[assignment_df['legal-entity-text_x'] == 'INDIV', 'party-name_y'].str.replace('|'.join(titles), '').str.strip()
assignment_df.loc[assignment_df['legal-entity-text_y'] == 'INDIV', 'party-name_x'] = assignment_df.loc[assignment_df['legal-entity-text_y'] == 'INDIV', 'party-name_x'].str.replace('|'.join(titles), '').str.strip()


<div class="alert alert-block alert-warning">

## 6. Export to JSON <a class="anchor" name="export"></a>

</div>

After a series of data cleaning and re-formatting to ensure the ideal formats are met, the finalised dataframe is converted back into dictionary. Within this dictionary, 'assignors-info' and 'assignees-info' are nested dictionaries, matching json file format.

Finally, this dictionary is exported to as JSON file using the `json.dump()`, with indentation added for better readability [3]. This ensures that the JSON output is formatted in a clear and organized form.

----

In [ ]:
# Convert into dict
result = {}
for _, row in assignment_df.iterrows():
    rf_id = row['rf-id']
    if rf_id not in result:
        result[rf_id] = {
            'last-update-date': row['last_update_date'],
            'conveyance-text': row['conveyance_text'],
            'correspondent-party': row['correspondent_party'],
            'assignors-info': [],
            'assignees-info': [],
            'property-count': str(row['property_count']),
        }

    result[rf_id]['assignors-info'].append({
        'party-name': row['party-name_x'],
        'date-acknowledge': row['date-acknowledge'],
        'execution-date': row['execution-date'],
        'country': row['nationality_x'],
        'legal-entity-text': row['legal-entity-text_x']
    })
    result[rf_id]['assignees-info'].append({
        'party-name': row['party-name_y'],
        'country': row['nationality_y'],
        'legal-entity-text': row['legal-entity-text_y']
    })

In [ ]:
# Slice the dictionary to get only the first 5 records
result_slice = dict(list(result.items())[:5])

# Print the sliced dictionary
print(result_slice)

-------------------------------------

In [ ]:
# Convert the dictionary to JSON format
json_result = json.dumps(result, indent=4)

# Print the JSON string
print(json_result)


In [ ]:
# re-import for format checking
with open('task1_015.json', 'w',encoding='utf-8') as json_file:
    json.dump(result, json_file, indent=4)

<div class="alert alert-block alert-warning">

## 7. Summary <a class="anchor" name="summary"></a>

</div>


In this task, we learned about the process of transforming XML format data stored in text file to a JSON format. Through various steps of data parsing, cleaning, and reformatting, we were able to convert the raw text data into a structured JSON format.

Techniques such as regular expressions were used to extract information from the data, while various cleaning steps were implemented to reduce inconsistencies in the dataset. Throughout the task, we faced challenges such as handling nested data, inconsistencies in data like country names. These challenges were handled through step by step data cleaning.

To sum up, we use `regular expression` as the first step to extract essential information into different dataframe, and combine different dataframe with `rf_id`. We then addressed inconsistencies in the `country` column to ensure that the values were standardized. Finally, we save the data into a specified `JSON` format.

From this task, we gained technical skills in data wrangling and transformation but also learned problem-solving techniques. By identifying problems, generating ideas and trial and error to test the approaches, we were able to solve the problem and archieve the goal.

-------------------------------------

<div class="alert alert-block alert-warning">

## 8. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 13/04/2024.

[2]<a class="anchor" name="ref-2"></a> Avoiding error from pd.to_datetime in pandas,
https://stackoverflow.com/questions/36692861/avoiding-error-from-pd-to-datetime-in-pandas, Accessed 13/04/2024.

[3]<a class="anchor" name="ref-2"></a> JSON encoder and decoder,
https://docs.python.org/3/library/json.html, Accessed 13/04/2024.

